## eda + prelim linear regression 
on acled and ucdp data

In [64]:
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, ElasticNetCV, MultiTaskElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import pandas as pd
import matplotlib as plt


In [43]:

acled_df = pd.read_csv('../data/acled/darfur.csv') ## 9316 entries, 32 cols
# print(acled_df.info()) 
# print(acled_df.columns)
# acled_df['FATALITIES']


# ucdp_df = pd.read_csv("../data/ucdp/darfur_113023.csv").drop('low_est', axis=1) ## 1669 entries, 48 cols
# # print(ucdp_df.info()) 
# print(ucdp_df.columns)
# ucdp_df.dropna(inplace=True)
# ucdp_df

In [127]:
data = acled_df
X = data.drop('FATALITIES', axis=1)
y = data['FATALITIES']

# data = ucdp_df
# X = data.drop('best_est', axis=1)
# y = data['best_est']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define categorical and numerical features
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Create transformers for numeric and categorical features
## could use Imputer() from sklearn.preprocessing
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create a column transformer to apply different preprocessing to different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a pipeline with the preprocessor and a linear regression classifier
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', Ridge(alpha=75))
])

# Train the model
model.fit(X_train, y_train)

# Evaluate the model (you can use appropriate metrics for your problem)
## 1 is the best 
print("Model score: %.3f" % model.score(X_test, y_test))


Model score: 0.154


In [128]:
## evaluating the model 
from sklearn.metrics import mean_squared_error

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 496.30949051859545


In [129]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")


R-squared: 0.15438342684671047


In [126]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for Ridge
param_grid = {'classifier__alpha': [60, 70, 71, 72, 73, 74, 75]}  # Define the alpha values you want to test

# Create a GridSearchCV object
grid_search = GridSearchCV(model, param_grid, cv=10, scoring='r2')  # cv is the number of folds for cross-validation

# Fit the GridSearchCV object
grid_search.fit(X_train, y_train)

# Get the best estimator
best_model = grid_search.best_estimator_

# Evaluate the best model
print("Best Model score: %.3f" % best_model.score(X_test, y_test))
print(f"Best alpha value: {best_model.named_steps['classifier'].alpha}")


Best Model score: 0.154
Best alpha value: 75


## raw results

on acled, darfur data:

test size: .2
- linear: LinearRegression()[0.1], Ridge() [0.147], Lasso()[0.052], ElasticNet()[0.05]
- trees: XGBRegressor()[0.25], RandomForestRegressor()[0.356, but takes a long time]
- neural network: MLPRegressor()[-5.803]
- also something called MultiTaskElasticNet() that doesn't work on sparse data 
- ElasticNetCV() does not run 

test size: .3
- LinearRegression() [0.304], Ridge() [0.300], Lasso and EN both [0.047]